1. Parse https://rusdisinfo.voxukraine.org/

In [355]:
import time
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.safari.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from django.core.validators import URLValidator
from django.core.exceptions import ValidationError

In [356]:
url = "https://rusdisinfo.voxukraine.org/"

In [357]:
def get_data(url):
    options = Options()
    options.add_argument("--headless")

    driver = webdriver.Safari(options=options)
    driver.get(url)

    page = driver.page_source
    soup = BeautifulSoup(''.join(page), 'html.parser')

    titles = soup.find_all('a')
    links = []

    start = "/narratives/"
    for title in titles:
        if title["href"].startswith(start):
            links.append(title["href"])
    
    df = pd.DataFrame(columns=['News', 'Fake'])
    
    count = 0
    for i, link in enumerate(links):
        driver.get(url + link)
        sub_page = driver.page_source
        sub_soup = BeautifulSoup(''.join(sub_page), 'html.parser')

        headers = sub_soup.find_all('h3', class_='Narrative_fakeLink___YbTe')

        for j, header in enumerate(headers):
            driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div[2]/div[2]/h3[{j+1}]').click()

            sub_page = driver.page_source
            sub_soup = BeautifulSoup(''.join(sub_page), 'html.parser')
            text = sub_soup.find_all('div', class_='Narrative_Debunking__gRBl1')
            fake_text = sub_soup.find_all('h3', class_='Narrative_fakeLink___YbTe')
            
            df.loc[count, 'News'] = text[0].text
            df.loc[count, 'Fake'] = 0
            df.loc[count+1, 'News'] = fake_text[0].text
            df.loc[count+1, 'Fake'] = 1
            
            count += 2
            driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div[2]/div[2]/h3[{j+1}]').click()
    
    driver.quit()
    
    return df

In [358]:
data = get_unfake_data(url)

In [359]:
data

,News,Fake
0,Not the Revolution of Dignity but Russian aggr...,0
1,Maidan led to the separation of Donetsk and Lu...,1
2,Such statements are unfounded. The Ukrainian a...,0
3,Maidan led to the separation of Donetsk and Lu...,1
4,"In fact, there are representatives of the extr...",0
...,...,...
133,Poroshenko has made division among churches in...,1
134,"Ukraine has created its own Orthodox Church, w...",0
135,Poroshenko has made division among churches in...,1
136,The Malaysian Boeing over Donbass was shot dow...,0


2. Write a function to parse news pages (choose your own media or try to make a general one)

In [360]:
def parse(link):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Safari(options=options)
    
    
    val = URLValidator()
    try:
        val(f'{link}')
    except:
        driver.quit()
        return ""
    
    driver.execute_script(f"window.location.href = '{link}'")

    time.sleep(7)

    page = driver.page_source
    soup = BeautifulSoup(''.join(page), 'html.parser')

    articles = soup.find_all('p')
    articles_text = []

    for article in articles:
        articles_text.append(article.text)

    driver.quit()
    return articles_text

In [361]:
news = parse("https://www.bbc.com/news")

In [362]:
news

['Two major quakes have hit south-eastern Turkey, the first 7.8-magnitude tremor killing some 1,500 people in the country, and hundreds more in northern Syria.',
 'The death toll continues to rise after two powerful tremors struck as people slept.',
 'Five people are thought to have died as a particularly bloody period between the two sides continues.',
 'A large Russian attack is likely to come from the east and south, Kyiv says.',
 'A Chinese spokeswoman says the aircraft is for civilian use, but "deviated" from its intended route.',
 'The 16-year-old was riding jet skis and swimming with friends when she was attacked, police say.',
 'A British woman tells the BBC how the influencer persuaded her to work for his webcam business.',
 "The novelist was stabbed at a talk in the US last year and says he's found it hard to write since.",
 'An evacuation order is in place within one-mile radius of the scene amid fears of flying shrapnel.',
 'The Premier League charges Manchester City with m